In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

titanic = sns.load_dataset('titanic')

feature_names = ['pclass', 'female', 'age', 'fare']
titanic['female'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic.dropna(subset=feature_names, inplace=True)  #891 para 714

X = titanic[feature_names].to_numpy()
y = titanic['survived'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=123)

print('Tamanho de X_train: ', X_train.shape)
print('Tamanho de X_test: ', X_test.shape)
print('Tamanho de y_train: ', y_train.shape)
print('Tamanho de y_test: ', y_test.shape)


Tamanho de X_train:  (535, 4)
Tamanho de X_test:  (179, 4)
Tamanho de y_train:  (535,)
Tamanho de y_test:  (179,)


3 - Modifique a classe ClassBin para que a rede tenha a seguinte arquitetura:

1. Camada de Entrada: Mantém as 4 features de entrada.
2. Primeira Camada Oculta: nn.Linear com 4 neurônios de entrada e 16 neurônios de saída, seguida por uma ativação ReLU.
3. Segunda Camada Oculta: nn.Linear com 16 neurônios de entrada e 8 neurônios de saída, seguida por uma ativação ReLU.
4. Camada de Saída: nn.Linear com 8 neurônios de entrada e 1 neurônio de saída, seguida por uma ativação Sigmoid.
5. Remova todas as camadas de Dropout para uma nova arquitetura.

In [ ]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.fc1 = nn.Linear(4, 16) # primeira hidden layer
        self.fc2 = nn.Linear(16, 8) # segunda hidden layer
        self.fc3 = nn.Linear(8, 1) # terceira hidden layer
        self.sigmoid = nn.Sigmoid() # output layer com ativação Sigmoid

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        
        return x

model = ClassBin()

6. Treine o novo modelo com os mesmos hiperparâmetros (épocas, taxa de aprendizado, etc.) e compare a acurácia final (de treino e teste) com a do modelo original.


In [12]:
loss_fn = nn.BCELoss()
epochs = 100
batch_size = 32  # X_train 535 / 32 = 16.71 (então são 17 batches de 32)
learning_rate = 0.1


In [13]:

# Instânciar o Otimizador Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Converter X e y para torch.Tensor
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

# Um Dataset de Tensores - Array [X, y]
train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, y_test)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

for t in range(epochs):
    model.train() # Colocar o modelo em modo de treinamento (calcula os gradientes)
    
    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]
        
        # Propagação (Feed Forward)
        y_pred = model(X)
    
        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))
        
        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")
    
model.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

train_pred = model(X_train)
train_pred = train_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model(X_test)
test_pred = test_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)


Época 1,
          Custo Treino: 0.797
Época 2,
          Custo Treino: 0.641
Época 3,
          Custo Treino: 0.749
Época 4,
          Custo Treino: 0.713
Época 5,
          Custo Treino: 0.708
Época 6,
          Custo Treino: 0.657
Época 7,
          Custo Treino: 0.716
Época 8,
          Custo Treino: 0.691
Época 9,
          Custo Treino: 0.716
Época 10,
          Custo Treino: 0.682
Época 11,
          Custo Treino: 0.717
Época 12,
          Custo Treino: 0.675
Época 13,
          Custo Treino: 0.634
Época 14,
          Custo Treino: 0.647
Época 15,
          Custo Treino: 0.674
Época 16,
          Custo Treino: 0.7
Época 17,
          Custo Treino: 0.667
Época 18,
          Custo Treino: 0.69
Época 19,
          Custo Treino: 0.721
Época 20,
          Custo Treino: 0.668
Época 21,
          Custo Treino: 0.7
Época 22,
          Custo Treino: 0.641
Época 23,
          Custo Treino: 0.602
Época 24,
          Custo Treino: 0.675
Época 25,
          Custo Treino: 0.682
Época 26,
    

In [14]:

print(f"Acurácia de Treino: {train_acc}")
print('\n ---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")

Acurácia de Treino: 0.590654194355011

 ---------------------------

Acurácia de Teste: 0.6033519506454468
